# Part 1: Genres and communities and plotting
---

---
# Part 2: TF-IDF to understand genres and communities
---

---
# Part 3: Sentiment of the artists and communities
---

In [12]:
import os
import re
import pandas as pd

# Load the LabMT word list into a dictionary for use in sentiment analysis
def load_labmt_wordlist(file_path):
    """
    Load the LabMT word list from a text file into a dictionary where words are keys
    and happiness scores are values.
    """
    column_names = ['word', 'happiness_rank', 'happiness_average', 'happiness_standard_deviation',
                    'twitter_rank', 'google_rank', 'nyt_rank', 'lyrics_rank']
    
    # Load the LabMT word list with the specified columns, skipping initial non-data lines
    labmt_data = pd.read_csv(file_path, sep='\t', skiprows=2, names=column_names)
    # Convert 'happiness_average' to numeric and drop non-numeric entries
    labmt_data['happiness_average'] = pd.to_numeric(labmt_data['happiness_average'], errors='coerce')
    labmt_data = labmt_data.dropna(subset=['happiness_average'])
    
    # Create a dictionary with words as keys and happiness scores as values
    happiness_dict = dict(zip(labmt_data['word'], labmt_data['happiness_average']))
    return happiness_dict

# Function to calculate sentiment score using LabMT word list
def calculate_sentiment(tokens, happiness_dict, neutral_threshold=5.0):
    """
    Calculate the sentiment score of a list of tokens using the LabMT word list.
    """
    scores = []
    for token in tokens:
        score = happiness_dict.get(token)
        # Only include words with scores that differ significantly from the neutral threshold
        if score and abs(score - neutral_threshold) > 1.0:
            scores.append(score)
    
    # Return the average sentiment score, or None if no valid words were found
    return sum(scores) / len(scores) if scores else None

# Function to tokenize the text by converting to lowercase and splitting by whitespace and punctuation
def tokenize_text(text):
    """
    Tokenize the input text into lowercase words, ignoring punctuation.
    """
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

# Function to calculate sentiment for each artist's page
def calculate_artist_sentiments(directory_path, happiness_dict):
    """
    Calculate the sentiment score for each text file in a directory and return a dictionary.
    """
    artist_sentiments = {}
    
    # Iterate through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  # Only process .txt files
            file_path = os.path.join(directory_path, filename)
            
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                
            # Tokenize the text
            tokens = tokenize_text(text)
            
            # Calculate sentiment score for the tokenized text
            sentiment_score = calculate_sentiment(tokens, happiness_dict)
            
            # Store sentiment score with filename (artist) as the key
            artist_sentiments[filename] = sentiment_score
    
    return artist_sentiments

# Specify the path to the LabMT word list file
file_path = 'labMIT-1.0.txt'

# Load the LabMT word list
happiness_dict = load_labmt_wordlist(file_path)

# Specify the directory path for country music performers
directory_path = 'country_music_performers'

# Calculate sentiment scores for each artist's page
artist_sentiments = calculate_artist_sentiments(directory_path, happiness_dict)

# Display the sentiment scores
artist_sentiments


{'Harry_McClintock.txt': 6.406433734939756,
 'Haley_&_Michaels.txt': 6.630117647058822,
 'Dickey_Betts.txt': 6.254839924670439,
 'Two_Tons_of_Steel.txt': 6.522873563218388,
 'Bacon_Brothers.txt': 6.98,
 'Cledus_T._Judd.txt': 6.329669966996701,
 'Charlie_Major.txt': 6.642797927461133,
 'Caryl_Mack_Parker.txt': 6.513076923076923,
 'Tenille_Arts.txt': 6.548525179856136,
 'Tyler_Hubbard.txt': 6.4588766519823695,
 'Roy_Rogers.txt': 6.433022813688221,
 'Steven_Lee_Olsen.txt': 6.601247524752475,
 'O._B._McClinton.txt': 6.268154506437763,
 'JT_Hodges.txt': 6.408661800486609,
 'Hank_Flamingo.txt': 6.366079545454543,
 'My_Darling_Clementine_(band).txt': 6.6899999999999995,
 'Shawn_Camp_(singer).txt': 6.370178041543019,
 'The_Dirt_Drifters.txt': 6.113571428571428,
 'Valerie_June.txt': 6.458049261083771,
 'Roxie_Dean.txt': 6.5459605911329986,
 'Warren_Zeiders.txt': 6.212918088737229,
 'Will_Hoge.txt': 6.290626151012896,
 'Osborne_Brothers.txt': 6.315682451253478,
 'Nick_Hoffman.txt': 6.64112781954

### Description of Sentiment Analysis Findings for Country Music Artist Profiles

Using the LabMT word list, we calculated sentiment scores for a collection of country music artist profiles. The function will:

1. Load the LabMT word list, mapping each word to its happiness score.
2. Process each token, checking if it exists in the LabMT list.
3. Sum the happiness scores for tokens that are present in the list and calculate an average score.


The sentiment scores of most wikis is around 6.1 to 6.9, with most profiles clustering around a slightly positive score near 6.4. 

This result aligns with typical patterns in general language sentiment, where LabMT scores often fall between 5 and 7 due to a natural linguistic tendency toward mild positivity. 

In this context, words reflecting everyday positivity, such as “good,” “music,” and “success,” tend to push sentiment scores just above 6, while strongly negative words are relatively rare. 

This suggests that the profiles generally employ favorable language, focusing on career highlights, achievements, and the appealing qualities of each artist. The slight variations within the 6.1–6.9 range indicate that while some profiles are more neutral, others emphasize more enthusiastic or celebratory descriptions. 



In [13]:
import numpy as np

# Calculate basic statistics for the sentiment scores
def calculate_statistics(artist_sentiments):
    """
    Calculate average, median, variance, 25th percentile, and 75th percentile of sentiment scores.
    """
    # Extract sentiment scores as a list, excluding None values
    sentiment_scores = [score for score in artist_sentiments.values() if score is not None]
    
    # Check if we have valid scores to calculate statistics
    if len(sentiment_scores) == 0:
        return {
            "average": None,
            "median": None,
            "variance": None,
            "25th_percentile": None,
            "75th_percentile": None
        }
    
    # Calculate statistics
    average_sentiment = np.mean(sentiment_scores)
    median_sentiment = np.median(sentiment_scores)
    variance_sentiment = np.var(sentiment_scores)
    percentile_25 = np.percentile(sentiment_scores, 25)
    percentile_75 = np.percentile(sentiment_scores, 75)
    
    # Return the statistics as a dictionary
    return {
        "average": average_sentiment,
        "median": median_sentiment,
        "variance": variance_sentiment,
        "25th_percentile": percentile_25,
        "75th_percentile": percentile_75
    }

# Calculate statistics for the sentiment scores
statistics = calculate_statistics(artist_sentiments)

# Display the results
statistics


{'average': 6.3995517311408285,
 'median': 6.413586206896536,
 'variance': 0.04319745655306958,
 '25th_percentile': 6.321995498874737,
 '75th_percentile': 6.497872340425527}

### Description of Average Sentiment across All The Pages

The average sentiment across all country music artist pages is approximately 6.4, indicating a generally positive tone in the content. 

The median is close to the mean, suggesting a symmetric distribution of sentiment scores, with most profiles slightly above neutral. The 25th and 75th percentiles (6.32 and 6.50, respectively) reveal a narrow interquartile range, meaning there is little variation between the sentiment of different profiles. 

This small variance (0.04) confirms that the profiles consistently exhibit a mild positivity, reflective of the favorable language often used in artist biographies.

This makes sense because when you read wikis yourself, they are often written in a similair tone.